In [51]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA 


In [52]:
df_train = pd.read_csv("train.csv")
df_train.shape
df_test = pd.read_csv('test.csv')

In [ ]:
y_train = df_train['y'].values

In [54]:
cols = [c for c in df_train.columns if 'X' in c]
print('Number of features: {}'.format(len(cols)))
print('Feature types:')
df_train[cols].dtypes.value_counts()

In [55]:
counts = [[], [], []]
for c in cols:
    typ = df_train[c].dtype
    uniq = len(np.unique(df_train[c]))
    if uniq == 1:
        counts[0].append(c)
    elif uniq == 2 and typ == np.int64:
        counts[1].append(c)
    else:
        counts[2].append(c)
    
print('Constant features: {} Binary features: {} Categorical features: {}\n'.format(*[len(c) for c in counts]))
print('Constant features:', counts[0])
print('Categorical features:', counts[2])

array([130.81,  88.53,  76.26, ..., 109.22,  87.48, 110.85])

In [56]:
Columns_new = list(set(df_train.columns) - set(['ID','y']))

Number of features: 376
Feature types:


int64     368
object      8
dtype: int64

In [57]:
y_train = df_train['y'].values
id_test = df_test['ID'].values
x_train = df_train[Columns_new]
x_test = df_test[Columns_new]

Constant features: 12 Binary features: 356 Categorical features: 8

Constant features: ['X11', 'X93', 'X107', 'X233', 'X235', 'X268', 'X289', 'X290', 'X293', 'X297', 'X330', 'X347']
Categorical features: ['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8']


In [58]:

def CHK(df):
    if df.isnull().any().any():
        print("no missing values")
    else:
        print("no missing values")
CHK(x_train)
CHK(x_test)

In [59]:
for column in Columns_new:
    cardinality = len(np.unique(x_train[column]))
    if cardinality == 1:
        x_train.drop(column, axis=1) # Column with only one
    # value is useless so we drop it
    x_test.drop(column, axis=1)
    if cardinality > 2: # Column is categorical
        mapper = lambda x: sum([ord(digit) for digit in x])
        x_train[column] = x_train[column].apply(mapper)
        x_test[column] = x_test[column].apply(mapper)
x_train.head()

In [60]:
x_train[cols].dtypes.value_counts()
n_comp = 12
pca = PCA(n_components=n_comp, random_state=20)
pca2_results_train = pca.fit_transform(x_train)
pca2_results_test = pca.transform(x_test)


no missing values
no missing values


In [61]:
import xgboost as xgb
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(pca2_results_train,y_train, test_size=0.2,random_state=35)

d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)
#d_test = xgb.DMatrix(x_test)
d_test = xgb.DMatrix(pca2_results_test)





C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,X361,X374,X224,X244,X10,X205,X348,X240,X261,X263,...,X34,X300,X168,X238,X308,X16,X255,X98,X151,X290
0,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,1,1,0,0,1,...,0,0,0,1,0,0,0,1,0,0
2,1,0,1,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,1,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [63]:


params = {}
params['objective'] = 'reg:linear'
params['eta'] = 0.02
params['max_depth'] = 4

def xgb_r2_score(preds, dtrain):
    labels = dtrain.get_label()
    return 'r2', r2_score(labels, preds)

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

clf = xgb.train(params, d_train,1000, watchlist, early_stopping_rounds=50, 
                feval=xgb_r2_score, maximize=True, verbose_eval=10)

# Predict your test_df values using XGBoost.

p_test = clf.predict(d_test)

sub = pd.DataFrame()
sub['ID'] = id_test
sub['y'] = p_test
sub.to_csv('xgb.csv', index=False)

sub.head()

[15:43:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[0]	train-rmse:99.04150	valid-rmse:98.70239	train-r2:-59.47071	valid-r2:-61.96931
Multiple eval metrics have been passed: 'valid-r2' will be used for early stopping.

Will train until valid-r2 hasn't improved in 50 rounds.
[10]	train-rmse:81.18310	valid-rmse:80.84457	train-r2:-39.62955	valid-r2:-41.24502
[20]	train-rmse:66.63157	valid-rmse:66.31034	train-r2:-26.36975	valid-r2:-27.42080
[30]	train-rmse:54.79031	valid-rmse:54.48565	train-r2:-17.50624	valid-r2:-18.18837
[40]	train-rmse:45.17240	valid-rmse:44.87848	train-r2:-11.57931	valid-r2:-12.01817
[50]	train-rmse:37.38147	valid-rmse:37.10155	train-r2:-7.61437	valid-r2:-7.89729
[60]	train-rmse:31.08475	valid-rmse:30.81077	train-r2:-4.95669	valid-r2:-5.13591
[70]	train-rmse:26.02225	valid-rmse:25.75129	train-r2:-3.17446	valid-r2:-3.28620
[80]	train-rmse:21.97581	

,ID,y
0,1,79.049286
1,2,96.350510
2,3,81.374931
3,4,77.107979
4,5,111.473167


In [64]:
percentiles = [98, 76.37, 55.55, 69, 88] 
first_subject = np.array(percentiles)
print (first_subject.dtype)

float64
